In [54]:
frames = []
data = []
labels = []
import librosa.core


with open('dadb\\Bed002.dadb','r') as file:
    line = file.readline()
    k=0
    while line:
        i = 0
        while line[i]!=',':
            i+=1
        begin_time = float(line[:i])
        begin_last = i
        
        i+=1
        while line[i]!=',':
            i+=1
        end_time = float(line[begin_last+1:i])
        
        while line[i] not in ['A','D','M','V','W','X','Y','Z']:
            i+=1
            if i>len(line)-1:
                print(line)
        i+=2
        words_w_times = line[i:].split('|')
        words = []
        for x in words_w_times[:-1]:
            words.append(x.split('+')[-1])
        
        last_item = words_w_times[-1].split(',')
        if '|' in line:
            words.append(last_item[0].split('+')[-1])
        else:
            words.append(words_w_times[0].split(',')[0].split('+')[-1])
            print(words)
            print(k)
        label = last_item[1]
        
        data.append(words)
        labels.append(label)
        
        frames.append(librosa.core.time_to_frames((begin_time,end_time)))
        line = file.readline()
        k+=1

['mah']
7
['yep']
9
['sibilance']
10
['sibilance']
11
['no']
14
['no']
16
['yeah']
25
['yeah']
26
['yeah']
28
['so']
29
['um']
36
['mmm']
38
['o_k']
40
['']
43
['o_k']
44
['']
46
['']
47
['']
48
['']
49
['thanks']
50
['um']
57
['um']
65
['um']
71
['or']
75
['oh']
78
['um']
84
['right']
87
['o_k']
89
['yeah']
96
['sure']
97
['right']
99
['right']
100
['o_k']
104
['mm-hmm']
105
['yep']
108
['right']
109
['o_k']
110
['right']
113
['o_k']
114
['o_k']
117
['right']
120
['right']
122
['right']
123
['mm-hmm']
125
['right']
127
['o_k']
130
['o_k']
173
['no']
175
['um']
178
['yeah']
185
['right']
186
['yep']
189
['input']
194
['yeah']
195
['yep']
198
['wow']
200
['organization']
201
['no']
203
['no']
204
['ah']
215
['o_k']
216
['um']
219
['mmm']
241
['um']
260
['afterwards']
284
['mm-hmm']
287
['o_k']
290
['yep']
291
['is']
295
['yeah']
300
['nuh']
328
['mm-hmm']
342
['yeah']
343
['what']
368
['o_k']
370
['huh']
371
['oh']
402
['yep']
408
['um']
413
['mm-hmm']
427
['yeah']
435
['o_k']
436
['o_k

In [48]:
def mapping_labels(label):
    if label=='' or label[0].isdigit():
        return ''
    if label[0] in ['s','q','b','%','f']:
        return label[0]
    if label[0] == 'h':
        return 'f'
    if label[0] == 'x':
        return '%'
    return label
set([mapping_labels(label) for label in labels])

{'', '%', 'b', 'f', 'q', 's', 'z'}

In [25]:
'l'.isdigit()

False

In [28]:
mapped = [mapping_labels(label) for label in labels]

In [49]:
words

['mmm,b.%,Bed002-c1,mn015,b.%,,,,,\n']